In [567]:
import pandas as pd
import numpy as np

# 1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.
## Acha a quantidade de linhas de detalhes (SalesOrderDetailID) por SalesOrderID.
- **Após fazer uma análise exploratória dos dados**, foi verificado que **para cada SalesOrderID existe uma SalesOrderDetailID.**
 - Foi feito um agrupamento da coluna SalesOrderID com a função de agregação size() para achar o número de linhas para cada SalesOrderID, **que é o mesmo número de linhas da coluna SalesOrderDetailID.**

In [568]:
def question_1(base1="Sales.SalesOrderDetail.csv"):
    
    # Importa base
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    
    # Agrupa SalesOrderDetail por SalesOrderID com a função de agregação size()
    grouped_SalesOrderDetail = SalesOrderDetail.groupby('SalesOrderID').size().reset_index(name='Entries')
    
    # Seleciona as linhas com número de registros maior ou igual a 3
    answer1 = grouped_SalesOrderDetail.loc[grouped_SalesOrderDetail["Entries"] >= 3]
    
    # Retorna o tamanho do dataframe answer, que é o resultado esperado
    return len(answer1)

In [569]:
question_1("Sales.SalesOrderDetail.csv")

12757

# 2. Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).
- **Após análise exploratória dos dados**, foi observado que a chave comum entre as 3 bases é a coluna **ProductID.**
- **Após análise exploratória dos dados**. verificou-se que as colunas relevantes para alcançar o resultado desejado são:
 - **OrderQty**, encontrada somente na tabela **Sales.SalesOrderDetail**;
 - **DaysToManufacture**, encontrada nas tabelas **Sales.SpecialOfferProduct e Production.Product**;
 - **Name**, encontrada nas tabelas **Sales.SpecialOfferProduct e Production.Product**.
- Foi feito um merge entre as tabelas SpecialOfferProduct e SalesOrderDetail para trazer a coluna OrderQty para a mesma tabela.
- Após, foi feito um merge com a abela Product utilizando como chave o ProductID.
 - O resultado foi agrupado por DaysToManufacture e Name com a função de agregação sum na coluna OrderQty.
 - Foram selecionados os 3 produtos mais vendidos.

In [570]:
def question_2(base1="Sales.SalesOrderDetail.csv", base2="Sales.SpecialOfferProduct.csv",
               base3="Production.Product.csv"):
    
    # Importa bases
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    SpecialOfferProduct = pd.read_csv(f"../data/{base2}", sep=";")
    Product = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Relaciona SpecialOfferProduct e Product para trazer os produtos a mais de SpecialOfferProduct
    SpecialOfferProduct_SalesOrderDetail = pd.merge(SpecialOfferProduct, SalesOrderDetail, on="ProductID", how="inner")
    
    # Relaciona Sales.SalesOrderDetail e Production.Product para trazer a coluna OrderQty
    Product_SpecialOfferProduct_SalesOrderDetail = pd.merge(Product, SpecialOfferProduct_SalesOrderDetail[["OrderQty", "ProductID"]], on="ProductID", how="inner")
    
    # Seleciona as colunas relevantes para diminuir o dataframe
    Product_SpecialOfferProduct_SalesOrderDetail = Product_SpecialOfferProduct_SalesOrderDetail[["ProductID", "Name", "OrderQty", "DaysToManufacture"]]
    
    # Agrupa o dataframe e aplica sum na coluna OrderQty
    Product_SpecialOfferProduct_SalesOrderDetail = Product_SpecialOfferProduct_SalesOrderDetail.groupby(["DaysToManufacture", "Name"]).agg({'OrderQty':sum,})
    
    # Seleciona os 3 produtos mais vendidos, retorna o nome agrupado por DaysToManufacture
    answer2 = Product_SpecialOfferProduct_SalesOrderDetail.groupby('DaysToManufacture').apply(lambda x: x.nlargest(3,"OrderQty")).reset_index(level=0, drop=True)
    
    return answer2

In [571]:
question_2()

OrderQty
DaysToManufacture Name                                    
0                 Sport-100 Helmet, Blue             33715
                  AWC Logo Cap                       33244
                  Sport-100 Helmet, Black            32660
1                 LL Road Frame - Black, 52           4743
                  ML Road Frame-W - Yellow, 44        4305
                  LL Road Frame - Red, 44             2973
2                 HL Mountain Frame - Silver, 38      2412
                  ML Road Frame-W - Yellow, 38         895
                  LL Mountain Frame - Silver, 40       884
4                 Mountain-200 Black, 38             11908
                  Road-650 Red, 44                    9016
                  Mountain-200 Black, 42              7992

# 3. Escreva uma query ligando as tabelas Person.Person, Sales.Customer e Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem de pedidos efetuados.
- **Após análise exploratória dos dados**, foi verificado que o nome dos cliente consta somente na tabela Person.Person.
- **Após análise exploratória**, foi identificado que as chaves **BusinessEntityID e CustomerID** são as mesmas
- Foi feito um relacionamento entre Customer e Person utilizando como chave, respectivamente, CustomerID e BusinessEntityID para trazer as colunas referentes ao nome para a mesma tabela.
- Para achar o número de pedidos realizados, basta calcular a quantidade de compras feitas por cada cliente.
 - **Após análise exploratória**, foi identificado que o campo **SalesOrderID** pode ser utilizado para calcular o número de comprar por cada cliente.
- Foi criada a coluna **Name**.
 - Feito agrupamento na coluna Name, aplicado a função size() na coluna **SalesOrderID** e organizado de forma decrescente.

In [572]:
def question_3(base1="Person.Person.csv", base2="Sales.Customer.csv",
               base3="Sales.SalesOrderHeader.csv"):
    
    # Importa bases
    Person = pd.read_csv(f"../data/{base1}", sep=";")
    Customer = pd.read_csv(f"../data/{base2}", sep=";")
    SalesOrderHeader = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Relacionamento entre Customer e Customer. Para o presente teste, não seria necessário.
    Person_Customer = pd.merge(Customer, Person, left_on="CustomerID", right_on="BusinessEntityID", how="inner")
    
    # Relacionamento entre SalesOrderHeader e Person_Customer para trazer o nome do cliente.
    SalesOrderHeader_Customer = pd.merge(SalesOrderHeader, Person_Customer[["FirstName", "MiddleName", "LastName", "CustomerID"]],
                               on="CustomerID", how="inner")
    
    # Cria a coluna Name
    SalesOrderHeader_Customer["Name"] = SalesOrderHeader_Customer["FirstName"]+" "+SalesOrderHeader_Customer["MiddleName"]+" "+SalesOrderHeader_Customer["LastName"]
    
    # Agrupa por Name e aplica a função de agregação size() na coluna SalesOrderID.
    SalesOrderHeader_Customer = SalesOrderHeader_Customer.groupby("Name")["SalesOrderID"].size().sort_values(ascending=False)
    
    # Transforma a série em um dataframe com a coluna NumberOfOrders.
    answer3 = SalesOrderHeader_Customer.to_frame(name="NumberOfOrders")
    
    return answer3

In [573]:
question_3()

,NumberOfOrders
Name,
Morgan C Miller,28
Grace R Lewis,27
Natalie M Martin,27
Grace J Lee,27
Marshall M Shen,27
...,...
Kaitlyn R Howard,1
Kaitlyn O Butler,1
Kaitlyn M Russell,1


# 4. Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e OrderDate.
- **Após análise exploratória dos dados**, verificou-se que o campo **OrderDate** consta na base **SalesOrderHeader** e que o campo **OrderQty** consta na base **SalesOrderDetail**.
- Feito relacionamento entre SalesOrderHeader e SalesOrderDetail para trazer os campos **OrderDate** e **OrderQty** para a mesma tabela.
- Após, foi feito um merge entre SalesOrderHeader_Detail e Product para relacionar os produtos.
- Para encontrar o resultado esperado, agrupou-se o dataframe por ProductID e OrderDate e aplicou-se a função de agregação sum na coluna OrderQty.

In [574]:
def question_4(base1="Sales.SalesOrderHeader.csv", base2="Sales.SalesOrderDetail.csv",
               base3="Production.Product.csv"):
    
    # Importa bases
    SalesOrderHeader = pd.read_csv(f"../data/{base1}", sep=";")
    SalesOrderDetail = pd.read_csv(f"../data/{base2}", sep=";")
    Product = pd.read_csv(f"../data/{base3}", sep=";")

    # Relaciona SalesOrderHeader e SalesOrderDetail para trazer as colunas OrderDate e OrderQty.
    SalesOrderHeader_Detail = pd.merge(SalesOrderHeader, SalesOrderDetail, on="SalesOrderID", how="inner")
    
    # Relaciona SalesOrderHeader_Detail com os Produtoprodutos.
    SalesOrderHeader_Detail_Product = pd.merge(SalesOrderHeader_Detail, Product[["ProductID"]], on="ProductID", how="inner")
    
    # Seleciona as colunas relevantes para diminuir o dataframe.
    SalesOrderHeader_Detail_Product = SalesOrderHeader_Detail_Product[["ProductID", 
                                                                       "OrderQty", "OrderDate"]]
    
    # Agrupa o dataframe e aplica sum na coluna OrderQty.
    answer4 = SalesOrderHeader_Detail_Product.groupby(["ProductID", "OrderDate"]).agg({'OrderQty':sum,})

    
    return answer4

In [575]:
question_4()

OrderQty
ProductID OrderDate                        
707       2011-05-31 00:00:00.000        24
          2011-07-01 00:00:00.000        58
          2011-08-01 00:00:00.000        55
          2011-08-31 00:00:00.000        41
          2011-10-01 00:00:00.000        77
...                                     ...
999       2014-05-25 00:00:00.000         3
          2014-05-27 00:00:00.000         1
          2014-05-28 00:00:00.000         3
          2014-05-29 00:00:00.000         1
          2014-05-30 00:00:00.000         1

[26878 rows x 1 columns]

# 5. Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido decrescente.
- Filtrou-se a coluna OrderDate somente pela data desejada (setembro de 2011)
- Filtrou-se a coluna TotalDue pelos valores superiores a 1000.

In [576]:
def question_5(base1="Sales.SalesOrderHeader.csv"):
    
    # Importa base
    SalesOrderHeader = pd.read_csv(f"../data/{base1}", sep=";", decimal=",")
    
    # Transforma a coluna OrderDate em datatime.
    SalesOrderHeader['OrderDate'] = pd.to_datetime(SalesOrderHeader['OrderDate'])
    
    # Selecionado o período em questão (setembro de 2011).
    SalesOrderHeader = SalesOrderHeader[(SalesOrderHeader["OrderDate"].dt.month == 9) & 
                                        (SalesOrderHeader["OrderDate"].dt.year == 2011)]
    
    # Transforma a coluna TotalDue em float para filtrá-la.
    SalesOrderHeader["TotalDue"] = SalesOrderHeader["TotalDue"].astype(np.float64)
    
    # Filtra os valores desejados (acima de 1000).
    answer5 = SalesOrderHeader[SalesOrderHeader["TotalDue"] > 1000].sort_values(by=["TotalDue"], ascending=False)
    
    return answer5

In [577]:
question_5()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
665,44324,8,2011-09-01,2011-09-13 00:00:00.000,2011-09-08 00:00:00.000,5,1,SO44324,NaN,10-4030-012267,12267,NaN,7,13133,13133,1,10780.0,930963Vi55635,1217.0,3578.27,286.2616,89.4568,3953.9884,NaN,0F60B3EE-0627-4A4B-832E-38C5514CE85C,2011-09-08 00:00:00.000
760,44419,8,2011-09-17,2011-09-29 00:00:00.000,2011-09-24 00:00:00.000,5,1,SO44419,NaN,10-4030-017352,17352,NaN,9,18298,18298,1,14483.0,135396Vi75107,1419.0,3578.27,286.2616,89.4568,3953.9884,NaN,7B16D224-CC2B-45E1-A92C-9EC4C9E48B38,2011-09-24 00:00:00.000
774,44433,8,2011-09-20,2011-10-02 00:00:00.000,2011-09-27 00:00:00.000,5,1,SO44433,NaN,10-4030-028270,28270,NaN,1,15033,15033,1,9433.0,931839Vi48642,NaN,3578.27,286.2616,89.4568,3953.9884,NaN,89FB6B95-ED37-4628-A27A-DDF876F2D68F,2011-09-27 00:00:00.000
773,44432,8,2011-09-20,2011-10-02 00:00:00.000,2011-09-27 00:00:00.000,5,1,SO44432,NaN,10-4030-014165,14165,NaN,10,11730,11730,1,157.0,931073Vi825,1465.0,3578.27,286.2616,89.4568,3953.9884,NaN,BDA72A99-CE09-4168-96C7-E17D8925ABC1,2011-09-27 00:00:00.000
772,44431,8,2011-09-19,2011-10-01 00:00:00.000,2011-09-26 00:00:00.000,5,1,SO44431,NaN,10-4030-017725,17725,NaN,9,27730,27730,1,3439.0,535409Vi18131,1445.0,3578.27,286.2616,89.4568,3953.9884,NaN,36683A9B-929B-48AF-8716-060DAFB01B34,2011-09-26 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,44378,8,2011-09-10,2011-09-22 00:00:00.000,2011-09-17 00:00:00.000,5,1,SO44378,NaN,10-4030-025903,25903,NaN,1,20717,20717,1,15388.0,333274Vi79476,NaN,3374.99,269.9992,84.3748,3729.3640,NaN,DF83A4AB-5947-4E6D-8F39-6413C2811026,2011-09-17 00:00:00.000
717,44376,8,2011-09-10,2011-09-22 00:00:00.000,2011-09-17 00:00:00.000,5,1,SO44376,NaN,10-4030-029399,29399,NaN,8,15257,15257,1,14705.0,431755Vi76218,1332.0,3374.99,269.9992,84.3748,3729.3640,NaN,AC88BACA-8A4C-4598-AB73-D88C8B888EDC,2011-09-17 00:00:00.000
678,44337,8,2011-09-04,2011-09-16 00:00:00.000,2011-09-11 00:00:00.000,5,1,SO44337,NaN,10-4030-011097,11097,NaN,9,22275,22275,1,7265.0,834144Vi37703,1250.0,3374.99,269.9992,84.3748,3729.3640,NaN,0838DBD4-443C-40B9-9125-1B20AF8CA06F,2011-09-11 00:00:00.000
705,44364,8,2011-09-08,2011-09-20 00:00:00.000,2011-09-15 00:00:00.000,5,1,SO44364,NaN,10-4030-011092,11092,NaN,9,15209,15209,1,2532.0,834140Vi13198,1302.0,3374.99,269.9992,84.3748,3729.3640,NaN,7E61F12A-49B1-45F5-BA89-F8054D47928B,2011-09-15 00:00:00.000


# Gráficos